In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

## The steps to get a DataFrame from one page of results look like this:
- • Build a URL by combining the base url with a specific page number
- • Use requests.post() to get the results of the post
- • Make a soup from results.text
- • Look at the soup to identify the table you want based on one of its attributes (like class)
- • Pass the table as a string to pandas read_html()
- • What does that look like? What is the datatype?
- • Keep working with the data until you have it a DataFrame

## Insert pagination values

In [2]:
pgs_2016 = 154
pgs_2017 = 147
pgs_2018 = 85
pgs_2019 = 113
pgs_half_2016 = 898
pgs_half_2017 = 892
pgs_half_2018 = 598
pgs_half_2019 = 690

## Base URLs for each race results page


In [3]:
urlbase_2019 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2019-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_2018 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2018-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_2017 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2017-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_2016 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2016-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2019 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2019-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2018 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2018-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2017 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2017-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='
urlbase_half_2016 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Half-Marathon/2016-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='

## Example pull of one page (99) of race data from 2019 Marathon

In [4]:
base = urlbase_2019
page = 99
url = base + str(99)
print(url)

https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2019-Results?gender=&agegroup=&bib=&firstname=&lastname=&page=99


In [5]:
response = requests.post(url)
print(type(response))
soup = BS(response.text, 'lxml')
print(type(soup))

<class 'requests.models.Response'>
<class 'bs4.BeautifulSoup'>


In [6]:
tables = soup.find_all('table', 
                       attrs = {'class': 'table table-responsive table-bordered'})

In [7]:
len(tables)

3

In [8]:
results_list = pd.read_html(str(tables[0])) ## returns a list of dataframes

In [9]:
len(results_list)

1

In [10]:
df = results_list[0]
df.shape

(25, 4)

In [11]:
df.head()

,Overall,Bib,Name,Time
0,99999,32379,Raquel Flores,00:00:00
1,99999,30292,Kyle Domingos,00:00:00
2,99999,32850,Paul Dillard,00:00:00
3,99999,31415,Nicole Bennett,00:00:00
4,99999,32995,Rudy Novak,00:00:00


## A test to see what tables[2] looks like...

In [12]:
pd.read_html(str(tables[2]))

[    Top Men                             
   Gender PL               Name      Time
 0         2     Scott Wietecha  02:34:59
 1         3      Jordan Wilson  02:35:24
 2         4     Steelton Flynn  02:39:59
 3         5       Thomas Ellis  02:42:09
 4         6   Nicholas Tseffos  02:48:42
 5         7  Satoshi Mitsumori  02:50:33
 6         8   Harrison Kieffer  02:51:18
 7         9       Steven Forte  02:54:34
 8        10         Grant Rice  02:55:49
 9        11      Andrew Fisher  02:56:05]

## Write a "test function" to itterate through all 85 race pages for 2018 Full

In [13]:
## urlbase_2018 = 'https://www.runrocknroll.com/Events/Nashville/The-Races/Marathon/2018-Results?gender=&agegroup=&bib=&firstname=&lastname=&page='

base = urlbase_2018
pgs_2018 = 85

column_names = ['Overall', 'Bib', 'Name', 'Time']

## Create empty dataframe
full_2018 = pd.DataFrame(columns= column_names) 


## Iterate over url_base2018        
for page in range (1, pgs_2018+1):
        
    base = urlbase_2018
    url = base + str(page)
    response = requests.post(url)
    soup = BS(response.text, 'lxml')
    tables = soup.find_all('table', 
                     attrs = {'class': 'table table-responsive table-bordered'})
    results_list = pd.read_html(str(tables[0]))
    full_2018 = full_2018.append(pd.DataFrame(results_list[0]))
    


In [14]:
full_2018.shape

(2115, 4)

In [15]:
df.head()

,Overall,Bib,Name,Time
0,99999,32379,Raquel Flores,00:00:00
1,99999,30292,Kyle Domingos,00:00:00
2,99999,32850,Paul Dillard,00:00:00
3,99999,31415,Nicole Bennett,00:00:00
4,99999,32995,Rudy Novak,00:00:00


## Define a generic function to read through all pages of each race.

In [16]:
def marathon_data_df(base, pages):
    """This function will pull all pages of race for each event"""

    column_names = ['Overall', 'Bib', 'Name', 'Time']

    ## Create empty dataframe
    df = pd.DataFrame(columns= column_names) 

    ## Iterate over all pages in race      
    for page in range (1, pages+1):
        
        url = base + str(page)
        response = requests.post(url)
        soup = BS(response.text, 'lxml')
        tables = soup.find_all('table', 
                     attrs = {'class': 'table table-responsive table-bordered'})
        results_list = pd.read_html(str(tables[0]))
        df = df.append(pd.DataFrame(results_list[0]))
    
    return df



## Create DataFrame for 2019 Full Marathon

In [17]:
full_2019 = marathon_data_df(urlbase_2019, pgs_2019)

KeyboardInterrupt: 

In [ ]:
full_2019.shape

In [ ]:
full_2019.head()

In [ ]:
type(full_2019)

## Create DataFrame for 2017 Full Marathon

In [ ]:
full_2017 = marathon_data_df(urlbase_2017, pgs_2017)

In [ ]:
full_2017.shape

In [ ]:
full_2017.head()

## Create DataFrame for 2016 Full Marathon

full_2016 = marathon_data_df(urlbase_2016, pgs_2016)

In [ ]:
full_2016.shape

In [ ]:
full_2016.head()

## Create Lists for the Half Marathons to use in Loop

In [ ]:
bases_half_list = [urlbase_half_2019, urlbase_half_2018, urlbase_half_2017, urlbase_half_2016]

In [ ]:
pages_half_list = [pgs_half_2019, pgs_half_2018, pgs_half_2017, pgs_half_2016]

In [ ]:
df_half_list = []
for base, pages in zip(bases_half_list, pages_half_list):
    df_half_list.append(marathon_data_df(base, pages))

## Convert dataframes to csv files

In [ ]:
df_names = ['half_2019.csv', 'half_2018.csv', 'half_2017.csv', 'half_2016.csv']
for df, csv_name in zip(df_half_list, df_names):
    df.to_csv(csv_name, index = False)

In [ ]:
full_2019.to_csv('full_2019.csv')

In [ ]:
full_2018.to_csv('full_2018.csv')

In [ ]:
full_2017.to_csv('full_2017.csv')
full_2016.to_csv('full_2016.csv')